<a href="https://colab.research.google.com/github/AndyOmondi17/CompilerConstruction/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Group Members: 105493, 101170, 112448**

**Lab Activity 1:Flex,Bison,Lex,Yacc**

**Flex(fast lexical analyzer generator)**-A tool/computer program for generating lexical analyzers (scanners or lexers) written by Vern Paxson in C around 1987.



**Bison**

**Lex**

**Yacc**